In [5]:
import duckdb
import pandas as pd
import os

In [6]:
con = duckdb.connect(database=':memory:')

df_duck = con.execute("""
    SELECT * FROM 'data/estimativa_cre.parquet'
    """).df()

print(df_duck)

IOException: IO Error: No files found that match the pattern "data/estimativa_cre.parquet"

In [7]:
from openai import OpenAI

# ========================
# 1. Conexão DuckDB e carregamento do Parquet
# ========================
con = duckdb.connect(database=":memory:")

# Cria tabela a partir do Parquet (nome lógico: estimativa_cre)
con.execute("""
    CREATE TABLE estimativa_cre AS 
    SELECT * FROM 'data/estimativa_cre.parquet'
""")

# DataFrame com preview das colunas (para enviar ao prompt)
df_duck = con.execute("SELECT * FROM estimativa_cre LIMIT 5").df()

# ========================
# 2. Conexão com modelo da HuggingFace
# ========================
client = OpenAI(
    base_url="https://router.huggingface.co/v1",
    api_key=os.getenv("HF_TOKEN"),
)

# ========================
# 3. Função de consulta via IA
# ========================
def consulta_ia(pergunta: str):
    prompt = f"""
    Você é um assistente que gera consultas SQL para DuckDB.
    A tabela disponível se chama 'estimativa_cre' e tem as seguintes colunas:
    {', '.join(df_duck.columns.tolist())}

    Pergunta do usuário: {pergunta}

    Responda apenas com a SQL.
    """

    response = client.chat.completions.create(
        model="meta-llama/Meta-Llama-3-8B-Instruct",
        messages=[{"role": "user", "content": prompt}],
        max_tokens=300,
        temperature=0,
    )

    query = response.choices[0].message.content.strip()

    # Executa a query no DuckDB
    try:
        resultado = con.execute(query).fetchdf()
        return query, resultado
    except Exception as e:
        return query, f"Erro ao executar SQL: {e}"

IOException: IO Error: No files found that match the pattern "data/estimativa_cre.parquet"

In [16]:
query, resultado = consulta_ia("O total arrecadado por ano da fonte 500")
print("Query usada:\n", query)
print("\nResultado:\n", resultado)

Query usada:
 SELECT ano, SUM(valor) AS total_arrecadado 
FROM estimativa_cre 
WHERE nova_fonte = 500 
GROUP BY ano

Resultado:
     ano  total_arrecadado
0  2023      8.622825e+09
1  2024      1.002981e+10
2  2025      1.069943e+10
3  2026      1.127532e+10
4  2027      1.216892e+10
